1. Euclidian distance function [4 points]
============
The squared Euclidean distance between two D-dimensional vectors x and z is given by
$\newcommand{\norm}[1]{\left\lVert#1\right\rVert}$

$\norm{\mathbf{x} - \mathbf{z}}_{2}^{2} = (\mathbf{x} - \mathbf{z})^\top(\mathbf{x} - \mathbf{z}) = \sigma_{d=1}^{D}(X_d - Z_d)^2$

Assume we have two groups of D-dimensional data points represented respectively by a $N_1 \times D$ matrix...

In [49]:
import tensorflow as tf;

    
a = tf.constant([1,2,3,4], name='a')
b = tf.constant([1,2,3,4], name='b')

D_euc = tf.tile(tf.expand_dims(a,1), [1,]2)

sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(D_euc))

[[1]
 [2]
 [3]
 [4]
 [1]
 [2]
 [3]
 [4]]
